<a href="https://colab.research.google.com/github/Rhodham96/LGG-Thomas5/blob/Robin/PersonalProjects/Transformers/Attention_All_I_Need/ShakespeareGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =======================================================
# Name: Hamers Robin
# GitHub: Rhodham96
# Year: 2025
# Description: Attention is all you need - Build a GPT from scratch, helped with Andrej Kartpathy video "Let's build GPT: from scratch, in code, spelled out"
# =======================================================

import sentencepiece as spm
import torch
import torch.nn as nn
from torch.nn import functional as F




In [2]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parrallel
block_size = 256 # what is the maximum context length for predictions ?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------------

torch.manual_seed(1337)

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding= 'utf-8') as f:
    text = f.read()

# here are all the tokens :
vocab_size = 1000
spm.SentencePieceTrainer.Train(f'--input=input.txt --model_prefix=spm --vocab_size={vocab_size}')
s = spm.SentencePieceProcessor(model_file='spm.model')
data = torch.tensor(s.encode(text, out_type=int, enable_sampling=True, alpha=0.1, nbest_size=-1), dtype=torch.long)

"""chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
data = torch.tensor(encode(text), dtype=torch.long)"""

# split train and val data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

--2025-03-28 13:50:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2025-03-28 13:50:44 (31.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# data loading
def get_batch(split):
    # generate a smal batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [5]:
@torch.no_grad()  # tells pytorch that everything in this function will not call backward it can be a lot more efficient
def estimate_loss():
    out = {}
    model.eval()
    for split in['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B,C,T) ---> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,T) @ (B,C,T) ---> (B,T,C)
        return out


In [7]:
class MultiHeadAttention(nn.Module):
    # multiple heads of self attention module

    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size)for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [8]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [9]:
class Block(nn.Module):
    """ Transformer block : communication followed by computation"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [10]:
# Decoder Only Transformer
class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and target are both (B,T), tensors of integers
        tok_embd = self.token_embedding_table(idx) #(B,T,C) = batch (4) * time (8) * channel (n_embd)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_embd + pos_embd # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size) C is vocab_size here

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #Pytorch expect (B,C,T)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} ")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(s.decode(m.generate(context, max_new_tokens=500)[0].tolist()))
"""print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))"""

11.507944 M parameters
step 0: train loss 7.1067, val loss 7.1000 
step 500: train loss 3.6500, val loss 3.7652 
step 1000: train loss 3.1461, val loss 3.4063 
step 1500: train loss 2.8230, val loss 3.2597 
step 2000: train loss 2.5796, val loss 3.1954 
step 2500: train loss 2.3634, val loss 3.1847 
step 3000: train loss 2.1561, val loss 3.2424 
step 3500: train loss 1.9469, val loss 3.3324 
step 4000: train loss 1.7384, val loss 3.4432 
step 4500: train loss 1.5474, val loss 3.5439 
 ⁇ ETER: There is not Nor here hearfulety to some expite. Yet, sirrant to't too be a can poor, even their own bloved in all that mocks. The hearers. Beseech we no tell you? lie to the business: O This then were him, the dear in haste, our temptory. Servanttion doth thou, commit in place, wert thou being one a man of a tlaw, how art thou wilt be condulity being contentivine, When vain things it may be about the longuish; and willt unframe the tainty-- After the very let an it up: let me not be 'laooks. Firs

'print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))'